# 1. Mount Google Drive

Run each session with Google Colab.

In [1]:
# Run each session with Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2. Setup Directories and git clone https://github.com/matterport/Mask_RCNN.git  

Run once.

In [ ]:
# Run once
%cd /content/drive/MyDrive/crane_pennsylvania
!mkdir /content/drive/MyDrive/crane_pennsylvania/mask-rcnn-repository

/content/drive/MyDrive/crane_pennsylvania


In [ ]:
# Run once
!mkdir /content/drive/MyDrive/crane_pennsylvania/predictions

In [ ]:
# Run once
!mkdir /content/drive/MyDrive/crane_pennsylvania/polys

In [ ]:
# Run once
%cd /content/drive/MyDrive/crane_pennsylvania/mask-rcnn-repository

/content/drive/MyDrive/crane_pennsylvania/mask-rcnn-repository


In [ ]:
# Run once
!git clone https://github.com/matterport/Mask_RCNN.git

fatal: destination path 'Mask_RCNN' already exists and is not an empty directory.


# 3. Install Mask R-CNN 
Run each session with Google Colab

In [ ]:
# Run each session with Google Colab
# Thanks to https://emadehsan.com/p/object-detection
%cd /content/drive/MyDrive/crane_pennsylvania/mask-rcnn-repository/Mask_RCNN
!python setup.py install 

/content/drive/MyDrive/crane_pennsylvania/mask-rcnn-repository/Mask_RCNN
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg

In [ ]:
!pip install 'h5py<3.0.0'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 8.2 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [ ]:
import os
import sys
# Root directory of the project
ROOT_D = os.path.abspath("./content/drive/MyDrive/crane_pennsylvania/mask-rcnn-repository/Mask_RCNN")

# Import Mask RCNN
sys.path.append(ROOT_D)

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import keras
import skimage
print(tf.__version__)
print(keras.__version__) # == 2.2.4
#Tensorflow-gpu version #== 1.12.0
print(skimage.__version__) # == 0.14.2


After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.


Using TensorFlow backend.


1.15.2
2.3.1
0.18.3


In [ ]:
# Write Annot
def write_annot(obj_type, obj_annots_dir, obj_f_num, org_f_name, org_f_path,org_f_width,org_f_height,org_f_depth, refPts, scores):
    # With credit to: 
    # https://www.geeksforgeeks.org/reading-writing-text-files-python/
    annot_file_path = obj_annots_dir+obj_f_num+'.xml'
    annot_file = open(annot_file_path,"w") 

    annot_file.write("<annotation>\n") 
    annot_file.write("	<folder>"+obj_type+"</folder>\n") 
    annot_file.write("	<filename>"+org_f_name+"</filename>\n") 
    annot_file.write("	<path>"+org_f_path+"</path>\n") 
    annot_file.write("	<source>\n") 
    annot_file.write("		<database>Muhlenberg_charcoal_hearths</database>\n") 
    annot_file.write("	</source>\n") 
    annot_file.write("	<size>\n") 
    annot_file.write("		<width>"+str(org_f_width)+"</width>\n") 
    annot_file.write("		<height>"+str(org_f_height)+"</height>\n") 
    annot_file.write("		<depth>"+str(org_f_depth)+"</depth>\n") 
    annot_file.write("	</size>\n")
    annot_file.write("	<!-- <object_present>0 = the object is not present in the image. 1 = it is -->\n")
    annot_file.write("	<object_present>0</object_present>\n")    
    for ocn in range(0,len(refPts)):
        refPt = refPts[ocn]
        refPtMin = refPt[0]
        refPtMax = refPt[1]
        annot_file.write("	<object>\n") 
        annot_file.write("		<name>"+obj_type+"</name>\n") 
        annot_file.write("		<number>"+str(ocn)+"</number>\n") 
        annot_file.write("		<score>"+str(scores[ocn])+"</score>\n")
        annot_file.write("		<correct>0</correct>\n")
        annot_file.write("		<bndbox>\n") 
        annot_file.write("			<xmin>"+str(refPtMin[0])+"</xmin>\n") 
        annot_file.write("			<ymin>"+str(refPtMin[1])+"</ymin>\n") 
        annot_file.write("			<xmax>"+str(refPtMax[0])+"</xmax>\n") 
        annot_file.write("			<ymax>"+str(refPtMax[1])+"</ymax>\n") 
        annot_file.write("		</bndbox>\n") 
        annot_file.write("	</object>\n") 
    annot_file.write("</annotation>\n") 
    annot_file.close()



# Quality Assurance: Confirm we have 598 slope tifs and 598 DEM slope jpgs with matching file names.

In [ ]:
"""
!mkdir /content/drive/MyDrive/crane_pennsylvania/predictions/project_pa_south_central_b2_2017/
!mkdir /content/drive/MyDrive/crane_pennsylvania/predictions/project_pa_south_central_b2_2017/xmls/
"""

mkdir: cannot create directory ‘/content/drive/MyDrive/crane_pennsylvania/predictions/project_pa_south_central_b2_2017/’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/crane_pennsylvania/predictions/project_pa_south_central_b2_2017/xmls/’: File exists


In [ ]:
import os
#batch_group = "pa_sandy_2014/"
batch_group = "pa_south_central_b2_2017"
dem_tif_fp = os.path.join("/content/drive/MyDrive/crane_pennsylvania/dem/",batch_group)
slope_tif_fp = os.path.join("/content/drive/MyDrive/crane_pennsylvania/slope/",batch_group)
slope_tif_tiles_fp = os.path.join(slope_tif_fp,"tiles/")
slope_tif_tiles_jpgs_fp = os.path.join(slope_tif_tiles_fp,"jpgs/")
prediction_fp = '/content/drive/MyDrive/crane_pennsylvania/predictions/project_'+batch_group+'/'
prediction_xmls_fp = '/content/drive/MyDrive/crane_pennsylvania/predictions/project_'+batch_group+'/xmls/'
#dem_tif_fp, slope_tif_fp, slope_tif_tiles_fp, slope_tif_tiles_jpgs_fp, prediction_xmls_fp

In [ ]:
import os

# Dictionary of slope_tifs

slope_tifs_dict = {}
for file_name in os.listdir(slope_tif_tiles_fp):
    origin_file_path = os.path.join(slope_tif_tiles_fp,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".tif"):
            # be careful comparing a partial filename
            slope_tifs_dict[file_name[:-4]]=file_name


# Dictionary of tifs
slope_jpgs_dict = {}
for file_name in os.listdir(slope_tif_tiles_jpgs_fp):
    origin_file_path = os.path.join(slope_tif_tiles_jpgs_fp,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".jpg"):
            # be careful comparing a partial filename
            slope_jpgs_dict[file_name[:-4]]=file_name
#print(tifs_dict)
print("Check that the batch group below is correct.")
print("There are: ",len(slope_tifs_dict)," tif files in ", slope_tif_tiles_fp)
print("There are: ",len(slope_jpgs_dict)," jpg files in ", slope_tif_tiles_jpgs_fp)
for dict_key in slope_tifs_dict:
    if not dict_key in slope_jpgs_dict:
        print(dict_key, " (tif) is not in jpgs directory.")

Check that the batch group below is correct.
There are:  11550  tif files in  /content/drive/MyDrive/crane_pennsylvania/slope/pa_south_central_b2_2017/tiles/
There are:  11550  jpg files in  /content/drive/MyDrive/crane_pennsylvania/slope/pa_south_central_b2_2017/tiles/jpgs/


In [ ]:

# Dictionary of slope_tifs

#dem_tif_fp, slope_tif_fp, slope_tif_tiles_fp, slope_tif_tiles_jpgs_fp

slope_tifs_dict = {}
for file_name in os.listdir(slope_tif_tiles_fp):
    origin_file_path = os.path.join(slope_tif_tiles_fp,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".tif"):
            # be careful comparing a partial filename
            slope_tifs_dict[file_name[:-4]]=file_name


# Dictionary of tifs
slope_jpgs_dict = {}
for file_name in os.listdir(slope_tif_tiles_jpgs_fp):
    origin_file_path = os.path.join(slope_tif_tiles_jpgs_fp,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".jpg"):
            # be careful comparing a partial filename
            slope_jpgs_dict[file_name[:-4]]=file_name
#print(tifs_dict)

print("There are: ",len(slope_tifs_dict)," tif files in ", slope_tif_tiles_fp)
print("There are: ",len(slope_jpgs_dict)," jpg files in ", slope_tif_tiles_jpgs_fp)
for dict_key in slope_tifs_dict:
    if not dict_key in slope_jpgs_dict:
        print(dict_key, " (tif) is not in jpgs directory.")

for dict_key in slope_jpgs_dict:
    if not dict_key in slope_tifs_dict:
        print(dict_key, " (jpg) is not in tifs directory.")

xmls_dict = {}
for file_name in os.listdir(os.path.join(prediction_fp)):
    origin_file_path = os.path.join(prediction_xmls_fp ,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".xml"):
            # be careful comparing a partial filename
            xmls_dict[file_name[:-4]]=file_name

print("There are: ",len(xmls_dict)," xml files in ", prediction_xmls_fp)
#for dict_key in xmls_dict:
#    if not dict_key in slope_jpgs_dict:
#        print(dict_key, " (xml) is not in jpgs directory.")

#print("There are: ",len(slope_jpgs_dict)," jpg files in ", slope_jpgs_fp)
for dict_key in slope_jpgs_dict:
    if not dict_key in xmls_dict:
        print(dict_key, " (jpg) is not in xmls directory.")


In [ ]:
# Dictionary of jpgs
slope_jpgs_dict = {}
for file_name in os.listdir(slope_tif_tiles_jpgs_fp):
    origin_file_path = os.path.join(slope_tif_tiles_jpgs_fp,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".jpg"):
            # be careful comparing a partial filename
            slope_jpgs_dict[file_name[:-4]]=file_name
print("There are: ",len(slope_jpgs_dict)," jpg files in ", slope_tif_tiles_jpgs_fp)

There are:  11550  jpg files in  /content/drive/MyDrive/crane_pennsylvania/slope/pa_south_central_b2_2017/tiles/jpgs/


In [ ]:
# Predict it for all the images
# Run Predict Config and Write Annot before running this cell.
cfg_name='cfg20200826T2315'
model_epoch='0016'
object_type = "charcoal_hearth_hill"

print("cfg_name:",cfg_name)

#For all of the jpegs, run prediction

# Run on all images - produce results for analysis.

import os
from os import listdir
import skimage
import cv2
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.config import Config
import mrcnn.model as modellib

# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = object_type+"_cfg" 
	# number of classes (background + class (charcoal hearth))
	NUM_CLASSES = 1 + 1
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

# create config
cfg = PredictionConfig()
# define the model

model = modellib.MaskRCNN(mode='inference', model_dir='/content/drive/MyDrive/MaskCNNhearths/code_and_working_data/charcoal_hearth_hill/model/', config=cfg)
print("Model loaded.")
# load model weights
model_path = '/content/drive/MyDrive/MaskCNNhearths/code_and_working_data/charcoal_hearth_hill/model/charcoal_hearth_hill_cfg20200826T2315/mask_rcnn_charcoal_hearth_hill_cfg_0016.h5'
model.load_weights(model_path, by_name=True)

#image_for_prediction_folder = images_dir

#Store the results in XML    
class_names = [object_type,object_type,]

# find all images
file_counter = 0
for filename in listdir(slope_tif_tiles_jpgs_fp):
    # extract image id
    image_id = filename[:-4]
    ext = filename[-4:]
    if(ext ==".jpg"):
        print(file_counter)  
        img_path = os.path.join(slope_tif_tiles_jpgs_fp, filename)
        #ann_path = annotations_dir + image_id + '.xml'
        #print(image_id,img_path,ann_path)
        print(image_id,img_path)
    
        image = cv2.imread(img_path)
        # Run detection

        results = model.detect([image], verbose=1)
        # Visualize results
        r = results[0]
        #print(r['rois'], class_names, r['scores'])
        #print(r['rois'][1],r['rois'][1][0],r['rois'][1][1],r['rois'][1][2],r['rois'][1][3])
        #image2= display_instances(image, r['rois'], r['masks'], r['class_ids'], 
        #class_names, r['scores'])
        obj_f_num = image_id
        refPts = []
    
        for rcn in range(0,len(r['rois'])):
            refPt = []
            refPtM = []
            refPtM.append(r['rois'][rcn][1])
            refPtM.append(r['rois'][rcn][0])
            refPt.append(refPtM)
            refPtM = []
            refPtM.append(r['rois'][rcn][3])
            refPtM.append(r['rois'][rcn][2])
            refPt.append(refPtM)
            refPts.append(refPt)
            #print(refPts)
        write_annot(class_names[0], prediction_xmls_fp, obj_f_num, filename, slope_tif_tiles_jpgs_fp ,image.shape[1],image.shape[0],image.shape[2], refPts,r['scores'])
        file_counter = file_counter + 1
print("The results are in: "+prediction_xmls_fp)

cfg_name: cfg20200826T2315
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Model loaded.
Re-starting from epoch 16
0
USGS_one_meter_x36y453_PA_South_Central_B2_2017r07c01 /content/drive/MyDrive/crane_pennsylvania/slope/pa_south_central_b2_2017/tiles/jpgs/USGS_one_meter_x36y453_PA_South_Central_B2_2017r07c01.jpg
Processing 1 images
image                    shape: (768, 1024, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32



Streaming output truncated to the last 5000 lines.
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
10836
USGS_one_meter_x35y454_PA_South_Central_B2_2017r00c00 /content/drive/MyDrive/crane_pennsylvania/slope/pa_south_central_b2_2017/tiles/jpgs/USGS_one_meter_x35y454_PA_South_Central_B2_2017r00c00.jpg
Processing 1 images
image                    shape: (768, 1024, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
10837
USGS_one_meter_x35y454_PA_South_Central_B2_2017r00c01 /content/drive/MyDrive/crane_pennsylvania/slope/pa_south_central_b2_2017/tiles/jpgs/USGS_one_meter_x35y454_PA_South_Central_B2_2017r00c01.jpg
Processing 1 images
image                    sh

In [ ]:
# run detection on 1 image
image = cv2.imread('XX/content/drive/MyDrive/crane_maryland/predictions/project_2021_nov_13/slope_tifs/slope_20120203_17SQD1896_utm.tif')
#image = skimage.io.imread(os.path.join(image_for_prediction_folder, test_file_names[cn]))
#print("shape:", image.shape)
# Run detection
print(image)
results = model.detect([image], verbose=1)